# 2. Cleaning Dataset

In [1]:
import pandas as pd
import datetime
from pandasql import sqldf
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

First I load the tables I converted from Yelp dataset.

In [2]:
coffee_business = pd.read_csv("Converted\yelp_business.csv")
coffee_tip = pd.read_csv("Converted\yelp_tip.csv")
coffee_review = pd.read_csv("Converted\yelp_review.csv")
coffee_user = pd.read_csv("Converted\yelp_user.csv")

Before working with the datasets, I verify all the data types of the attributes.

In [3]:
coffee_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438009 entries, 0 to 438008
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   438009 non-null  object
 1   review_id     438009 non-null  object
 2   user_id       438009 non-null  object
 3   review_stars  438009 non-null  int64 
 4   useful        438009 non-null  int64 
 5   funny         438009 non-null  int64 
 6   cool          438009 non-null  int64 
 7   text          438009 non-null  object
 8   date          438009 non-null  object
dtypes: int64(4), object(5)
memory usage: 30.1+ MB


I saw that the dates do not have the correct type. Before working with the dataset, I modify every feature with a date to DateTime. 

In [4]:
coffee_tip['date'] = pd.to_datetime(coffee_tip['date'])
coffee_review['date'] = pd.to_datetime(coffee_review['date'])
coffee_user['yelping_since'] = pd.to_datetime(coffee_user['yelping_since'])

In [5]:
coffee_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438009 entries, 0 to 438008
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   business_id   438009 non-null  object        
 1   review_id     438009 non-null  object        
 2   user_id       438009 non-null  object        
 3   review_stars  438009 non-null  int64         
 4   useful        438009 non-null  int64         
 5   funny         438009 non-null  int64         
 6   cool          438009 non-null  int64         
 7   text          438009 non-null  object        
 8   date          438009 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 30.1+ MB


The hour's column in the business table contains the opening and closing hours of each coffee shop by day of the week. To make the dataset tidy, I will create a new table using this attribute called hours. That table will have 14 attributes, one for the opening hour and the closing hour of each day. 

In [6]:
# The function receives a string and verifies that the hour has the correct format. 
# The time format is changed to an int as hh.mm
def timeFormat(hour):
    for i in range(len(hour)):
        if i == 0 or i == 1:
            if not hour[i].isnumeric():
                hour = '0' + hour
    h = datetime.datetime.strptime(hour,'%H:%M').time()
    return h.hour + h.minute/100  

In [7]:
# The function receives a string with key: value pairs where the key is the day of the week and the value an hour.
# It returns the opening and closing hours ordered by the day of the week.
def transformToWeekDays(hours):
    week = []
    for day in hours.split(','):
        l = day.split()
        n = len(l[1])
        if l[1][n-1] == '}': 
            n = n-1
        hours = l[1][1:n-1]
        hours = hours.split('-')
        for hour in hours:
            h = timeFormat(hour)
            week.append(h)
    return week

In [8]:
# Construction of the hours table. 
# Format of the hour hh.mm
coffee_weekdays = []
for index, row in coffee_business.iterrows():
    if type(row['hours']) is float or row['is_open'] == 0: 
        coffee_weekdays.append([None, None, None, None, None, None, None])
    else:
        coffee_weekdays.append(transformToWeekDays(row['hours']))

coffee_hours = pd.DataFrame(coffee_business['business_id'])
coffee_hours = coffee_hours.join(pd.DataFrame(coffee_weekdays, index=coffee_business.index, 
                          columns = ["Monday_open", "Monday_close", "Tuesday_open", "Tuesday_close",
                                     "Wednesday_open", "Wednesday_close", "Thursday_open", "Thursday_close",
                                     "Friday_open", "Friday_close", "Saturday_open", "Saturday_close",
                                     "Sunday_open", "Sunday_close"]))

coffee_hours

,business_id,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
0,DCsS3SgVFO56F6wRO_ewgA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_xOeoXfPUQTNlUAhXl32ug,5.3,23.0,5.3,23.0,5.3,23.0,5.3,23.0,5.3,23.0,6.3,23.0,6.3,23.0
2,lK-wuiq8b1TuU7bfbQZgsg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8k62wYhDVq1-652YbJi5eg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8Hvp1tYKiQbBgGIwkCRK5g,6.0,21.0,6.0,21.0,6.0,21.0,6.0,13.0,6.0,21.0,6.0,21.0,6.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8440,_rZyr1lrIoBaz65XiDPP6A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8441,NeM7anGnTOTn7sEJavS3sw,5.0,21.3,5.0,21.3,5.0,21.3,6.0,22.0,5.0,22.0,5.0,22.0,6.0,21.0
8442,00liP5s4IKsq97EH4Cc0Tw,4.0,20.0,4.0,20.0,4.0,20.0,4.0,21.0,5.0,21.0,5.0,19.3,4.0,20.0
8443,7V82ANZ7_ARkA7o0pAMAlA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I will remove the attributes and hours columns that will not be necessary for the analysis. 

In [9]:
del coffee_business['attributes']
del coffee_business['hours']
coffee_business_cleaned = coffee_business
coffee_business_cleaned.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,DCsS3SgVFO56F6wRO_ewgA,Missy Donuts & Coffee,1255 W Main St,Mesa,AZ,85201,33.414409,-111.858378,2.5,7,0,"Donuts, Juice Bars & Smoothies, Food, Coffee & Tea"
1,_xOeoXfPUQTNlUAhXl32ug,Starbucks,150 Boulevard Crémazie E,Montréal,QC,H2P 1E2,45.542993,-73.640218,3.5,4,1,"Coffee & Tea, Food"
2,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,NaN,Cleveland,OH,44113,41.489343,-81.711029,3.0,4,1,"Shopping Centers, Food, Coffee & Tea, Cafes, Museums, Restaurants, Shopping, Local Flavor, Flowers & Gifts, Arts & Entertainment, Art Galleries, Florists"
3,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Restaurants, Bakeries"
4,8Hvp1tYKiQbBgGIwkCRK5g,Tony's Family Restaurant,1515 W Pleasant Valley Rd,Parma,OH,44134,41.361185,-81.688755,4.0,60,1,"Coffee & Tea, Restaurants, Food, Breakfast & Brunch, American (Traditional)"


For the hour analysis, I need businesses that have all their schedule for the week. Therefore, I remove those with null values and that their open hour is the same as their closing hour for all days of the week. 

In [10]:
coffee_hours_cleaned  = sqldf("""
                    SELECT 
                        * 
                    FROM 
                        coffee_hours
                    WHERE 
                        Monday_open IS NOT NULL AND 
                        Tuesday_open IS NOT NULL AND 
                        Wednesday_open IS NOT NULL AND 
                        Thursday_open IS NOT NULL AND 
                        Friday_open IS NOT NULL AND 
                        Saturday_open IS NOT NULL AND 
                        Sunday_open IS NOT NULL
                    AND
                        (Monday_open <> Monday_close OR Tuesday_open <> Tuesday_close OR
                        Wednesday_open <> Wednesday_close OR Thursday_open <> Thursday_close OR
                        Friday_open <> Friday_close OR Saturday_open <> Saturday_close OR
                        Sunday_open <> Sunday_close) 
                        
                    """)
coffee_hours_cleaned.head()

,business_id,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
0,_xOeoXfPUQTNlUAhXl32ug,5.3,23.0,5.3,23.0,5.3,23.0,5.3,23.0,5.3,23.0,6.3,23.0,6.3,23.0
1,8Hvp1tYKiQbBgGIwkCRK5g,6.0,21.0,6.0,21.0,6.0,21.0,6.0,13.0,6.0,21.0,6.0,21.0,6.0,20.0
2,NLaK58WvlNQdUunSIkt-jA,0.0,0.0,7.3,17.0,7.3,17.0,7.3,17.0,7.3,17.0,8.3,17.0,8.3,17.0
3,1aVqiz43klXaFJUUx0H5fw,5.0,23.0,5.0,23.0,5.0,23.0,5.0,23.0,5.0,23.0,5.0,23.0,5.0,23.0
4,CfwrsG76Wm4iLS22v_wAcg,6.0,23.0,6.0,23.0,6.0,23.0,6.0,23.0,6.0,23.0,6.0,23.0,6.0,23.0


At last, I compare the original size of the hour table with the cleaned one and save all in new CSV files. 

In [11]:
print("          Original   Cleaned")
print("Hours:     ", len(coffee_hours),"     ", len(coffee_hours_cleaned))

          Original   Cleaned
Hours:      8445       4050


In [12]:
csv_name = "yelp_business_cleaned.csv"
coffee_business_cleaned.to_csv(csv_name, index=False)
csv_name = "yelp_tip_cleaned.csv"
coffee_tip.to_csv(csv_name, index=False)
csv_name = "yelp_review_cleaned.csv"
coffee_review.to_csv(csv_name, index=False)
csv_name = "yelp_user_cleaned.csv"
coffee_user.to_csv(csv_name, index=False)
csv_name = "yelp_hours_cleaned.csv"
coffee_hours_cleaned.to_csv(csv_name, index=False)